Input/output
============

Requirements
------------



This libraries have to be installed on your environment.

In [ ]:
import h5py
import silx
import fabio

EDF using FabIO
===============

For simple access, or fast access to raster format like EDF, MSK, TIFF... we recommand to use fabio `fabio`.

Reading files
-------------

In [ ]:
import fabio

image = fabio.open("data/medipix.edf")

# here is the data as a numpy array
image.data
print("Image size:", image.data.shape)

# here is the header as key-value dictionary
image.header
print("Metadata count:", len(image.header))
print("Metadata names:", list(image.header.keys()))

None

Play with it. You can see, it is easy to access to the image data, it is easy to access to simple metadata like *pixel size*. It is more difficult to access to motors or counters.

Writing files
-------------

In [ ]:
import fabio
import numpy

image = numpy.random.rand(10, 10)
metadata = {'pixel_size': '0.2'}

image = fabio.edfimage.edfimage(data=image, header=metadata)
image.write('new.edf')

Convert files
---------------------

In [25]:
import fabio

image = fabio.open('data/medipix.edf')
image = image.convert('tif')
image.save('filename.tif')

In [26]:
ls -al filename.*

-rw-r--r-- 1 valls valls 132951 Sep 18 16:11 filename.tif


HDF5 using h5py
===============

Read example
------------

In [27]:
import h5py

h5file = h5py.File('data/test.h5')

# print available names at the first level
print("First children:", list(h5file['/'].keys()))

# reaching a dataset from a sub group
dataset = h5file['/diff_map_0000/data/map']

# using size and types to not read the full stored data
print("Dataset:", dataset.shape, dataset.size, dataset.dtype)

# datasets mimics numpy-array
# read and apply the operation
a = 2 * dataset[0, 5]
# copy the data and store it as a numpy-array
b = dataset[...]

First children: ['diff_map_0000', 'diff_map_0001', 'diff_map_0002', 'diff_map_0003', 'diff_map_0004']
Dataset: (29, 78, 100) 226200 float32


Write example
-------------

In [ ]:
import numpy
import h5py

data = numpy.arange(10000.0)
data.shape = 100, 100

# write
h5file = h5py.File('my_first_one.h5', mode='w')

# write data into a dataset from the root
h5file['/data1'] = data

# write data into a dataset from group1
h5file['/group1/data2'] = data

h5file.close()

Using silx io
=============

`silx.io` provides a common API to read spec files, EDF, TIFF, and h5py files. This API is a read-only `h5py`-like.

Tool
-----------------

`h5ls` allow you to display the tree contained into an HDF5 file.

In [ ]:
import silx.io
import silx.io.utils

h5file = silx.io.open('data/test.h5')

string = silx.io.utils.h5ls(h5file)
print(string)

Read spec files using silx
--------------------------

In [44]:
import silx.io
data = silx.io.open('data/oleg.dat')

# print available scans
print("First childs:", data['/'].keys())

# print available measurements from the scan 94.1
print("Containt of measurement:", data['/94.1/measurement'].keys())

# get data from measurement
xdata = data['/94.1/measurement/Epoch']
ydata = data['/94.1/measurement/bpmi']
for row in zip(xdata, ydata):
    print(row)

First childs: ['94.1', '95.1', '96.1']
Containt of measurement: ['delta', 'H', 'K', 'L', 'Epoch', 'Seconds', 'Detector', 'Ion_m1', 'Ion_m2', 'srcur', 'curratt', 'ratio', 'all1', 'psd1', 'dir1', 'refl1', 'yoneda1', 'ACEdet', 'mcaLt1', 'twago', 'bpmi', 'tlangm', 'vO2', 'apdcnt', 'apdtemp', 'Monitor', 'detcorr', 'mca_0']
(2011829.0, 6.2475021e-07)
(2011831.0, 6.2534571e-07)
(2011833.0, 6.2587151e-07)
(2011836.0, 6.258831e-07)
(2011838.0, 6.2555091e-07)
(2011840.0, 6.2535531e-07)
(2011842.0, 6.2575771e-07)
(2011844.0, 6.2563299e-07)
(2011846.0, 6.2574901e-07)
(2011848.0, 6.2576453e-07)
(2011851.0, 6.2569779e-07)
(2011853.0, 6.2594449e-07)
(2011855.0, 6.2583979e-07)
(2011857.0, 6.2573582e-07)
(2011859.0, 6.2581711e-07)
(2011861.0, 6.257585e-07)
(2011863.0, 6.2580352e-07)
(2011866.0, 6.2588401e-07)
(2011868.0, 6.2564612e-07)
(2011870.0, 6.2587151e-07)
(2011872.0, 6.2575191e-07)


Convert spec file to HDF5
-------------------------

In [34]:
from silx.io.spectoh5 import write_spec_to_h5

write_spec_to_h5('data/oleg.dat', 'oleg.h5', mode='w')

In [35]:
ls -al oleg.*

-rw-r--r-- 1 valls valls 421152 Sep 18 16:14 oleg.h5


Read EDF file using silx
------------------------

In [43]:
import silx.io
data = silx.io.open('data/ID16B_diatomee.edf')

# Access to the frames
frames = data['/scan_0/instrument/detector_0/data']
len(frames)  # number of frames
frames[0]    # first frame
print("Number of frames:", len(frames))
print("Size of an image:", frames[0].shape)

# Access to motors, monitor, timestanp
srot = data['scan_0/instrument/positioners/srot'][...]
mon = data['scan_0/measurement/mon'][...]
timestamp = data['scan_0/instrument/detector_0/others/time_of_day'][...]
for row in zip(timestamp, srot, mon):
    print(row)

Number of frames: 6
Size of an image: (540, 640)
(1465802989.9281001, 0.0, 0)
(1465803040.360028, 14.371199607849121, 0)
(1465803090.7809851, 28.742399215698242, 0)
(1465803140.6969931, 42.969887999999997, 0)
(1465803191.6208999, 57.484798431396484, 0)
(1465803266.2914319, 71.856002807617188, 0)


Read HDF5 using silx
--------------------

For conveniance, ``silx`` also provides the h5py API for HDF5 files.

In [49]:
import silx.io

h5file = silx.io.open('data/test.h5')

# print available names at the first level
print("First children:", list(h5file['/'].keys()))

# reaching a dataset from a sub group
dataset = h5file['/diff_map_0000/data/map']

# using size and types to not read the full stored data
print("Dataset:", dataset.shape, dataset.size, dataset.dtype)

# datasets mimics numpy-array
# read and apply the operation
a = 2 * dataset[0, 5]
# copy the data and store it as a numpy-array
b = dataset[...]

First children: ['diff_map_0000', 'diff_map_0001', 'diff_map_0002', 'diff_map_0003', 'diff_map_0004']
Dataset: (29, 78, 100) 226200 float32


Exercise
========

1. Read the EDF file ``medipix.edf``.
2. Process the data

   - Create a mask for all the values below 10%.
   - With the above mask, set the affected pixels to 10%.
   - Optionally do the same for values above 90%.
   - This clamp values between 10% and 90%

3. Store the source, the mask of changed pixels and the result inside ``process.h5``, as below.

   ![Output file structure](images/exercise-result.png)

4. Load ``process.h5`` and list the root content


In [ ]:
# Load data/medipix.edf
# ...

# Process the data
# ...

# Load process.h5 to write inside: the source, the mask and the result
# ...